# kNN (k-nearest neighbors)

<img src="kNN.png"/>

## Ventajas
* Alta precisión
* Insensible a valores atípicos
* Sin suposiciones sobre datos

## Desventajas
* Computacionalmente caro
* Requiere mucha memoria

## Como funciona
> Con un conjunto de datos etiquetados
> que al recibir un dato sin etiqueta, se compara con los datos existentes, se
> toman los datos mas similares (los vecinos mas cercanos) usualmente es menor a 20, de aqui es donde viene k
> y se toman las etiquetas,
> se hace un conteo de estas y la que tiene mayor aparición es la que se le asigna al nuevo dato

In [ ]:
from numpy import * 
import operator

In [ ]:
def classify0(inX, dataSet, labels, k):
    #Calcula distancia del punta a todos los datos y los ordena de menor a mayor
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize,1)) - dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sortedDistIndicies = distances.argsort()
    classCount={}
    #Conteo de etiqutes y devuelve la etiqueta con mayor conteo
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

# Ejemplo: Clasificación de películas

Nombre Película | # de patadas | # de besos | Tipo de película
-- | -- | -- | -- 
California Man | 3 | 104 | Romance
He's Not Really into Dudes | 2 | 100 | Romance
Beautiful Woman | 1 | 81 | Romance
Kevin Longblade | 101 | 10 | Acción
Robo Slayer 3000 | 99 | 5 | Acción
Amped | 98 | 2 | Acción
? | 18 | 90 | No sabemos 



In [ ]:
datos = array([[3, 104], [2, 100], [1, 81], [101, 10], [99, 5], [98, 2]])
etiquetas = ['Romance', 'Romance', 'Romance', 'Acción', 'Acción', 'Acción']

In [ ]:
classify0([18,90], datos, etiquetas, 5)

### Distancia Euclidiana
Recordemos que la distancia euclidiana de un punto $A=(x1,y1)$ a otro punto $B=(x2,y2)$ esta dada por

$$d(A,B)=||A-B||=\sqrt{(x1-y1)^2+(x2-y2)^2}$$

# Ejemplo: Mejora de resultados de un sitio de citas 

Una persona ha estado usando algunos sitios de citas en línea para encontrar diferentes personas para salir. Se dio cuenta de que no le gustaba del todo las recomendaciones del sitio. Después de una cierta introspección, se dio cuenta de que había tres tipos de personas con las que salía:

* Gente a la que no le gustaba
* Gente a la que le gustaba en pequeñas dosis
* Gente a la que le gustaba en grandes dosis

Nos ha pedido que la ayudemos a filtrar futuros partidos para categorizarlos. Además, ha recopilado algunos datos que no están registrados por el sitio de citas, pero siente que es útil para seleccionar a la gente con la que sale.

Los que ha recopilado se encuentran en un archivo de texto llamado datingTestSet.txt. Ha estado recopilando estos datos por un tiempo y tiene 1000 entradas. Se han registrado las características siguientes:

* Número de millas de viajero frecuente ganadas por año
* Porcentaje de tiempo dedicado a jugar videojuegos
* Litros de helado consumido por semana

In [ ]:
#Muestra del archivo de texto
!head -4 datingTestSet.txt

### Cambio de archivo a matriz
Devuelve la matriz con los datos del archivo y el vector de etiquetas

In [ ]:
def file2matrix(filename):
    fr = open(filename)
    numberOfLines = len(fr.readlines())         #Numero de lineas en el archivo                                                                                   
    returnMat = zeros((numberOfLines,3))        #Inicializa la matriz a regresar                                                                                              
    classLabelVector = []                       #Inicializa el vector de las etiquetas                                                                                                 
    fr = open(filename)
    index = 0
    for line in fr.readlines():
        line = line.strip()
        listFromLine = line.split('\t')
        returnMat[index,:] = listFromLine[0:3]
        classLabelVector.append(listFromLine[-1])
        index += 1
    return returnMat,classLabelVector

In [ ]:
datosMatriz,datosEtiquetas = file2matrix('datingTestSet.txt')

In [ ]:
datosMatriz

### Visualización de los Datos 

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.plotting import figure, show

colormap = {'didntLike': 'red', 'smallDoses': 'green', 'largeDoses': 'blue'}
colors = [colormap[x] for x in datosEtiquetas]

p = figure(title = "Visualización de datos")
p.xaxis.axis_label = 'Millas de viajero frecuente'
p.yaxis.axis_label = 'Porcentaje dedicado a videojuegos'

p.circle(datosMatriz[:,0], datosMatriz[:,1],
         color=colors, fill_alpha=0.2, size=10)

show(p)

### Normalizar Datos

In [ ]:
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = zeros(shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - tile(minVals, (m,1))
    normDataSet = normDataSet/tile(ranges, (m,1))   #element wise divide                                                                                               
    return normDataSet, ranges, minVals

In [ ]:
normMat, ranges, minVals = autoNorm(datosMatriz)


In [ ]:
normMat

### Evaluación de precisión del algoritmo 
Vamos a medir el rendimiento del clasificador con la tasa de error, donde la tasa de error es el número de datos mal clasificadas dividido por el número total de datos probados.
Una tasa de error de 0 significa que tiene un clasificador perfecto, y una tasa de error de 1,0 significa que el clasificador siempre es incorrecto.

In [ ]:
def datingClassTest():
    hoRatio = 0.50      #hold out 10%                                                                                                                                  
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')       #load data setfrom file                                                                       
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],9)
        print("the classifier came back with: %s, the real answer is: %s" % (classifierResult, datingLabels[i]))
        if (classifierResult != datingLabels[i]): errorCount += 1.0
    print("the total error rate is: %f" % (errorCount/float(numTestVecs)))
    print(errorCount)

In [ ]:
datingClassTest()

# Uso de un clasificador

In [ ]:
def classifyPerson():
    percentTats = float(input("¿Porcentaje de tiempo dedicado a jugar juegos de video?"))
    ffMiles = float(input("¿Millas de viajero frecuente ganadas por año?"))
    iceCream = float(input("¿Litros de helado consumido por semana?"))
    DataMat,Labels = file2matrix('datingTestSet.txt')
    normaMat, rangos, minimosVals = autoNorm(DataMat)
    inArr = array([ffMiles, percentTats, iceCream])
    classifierResult = classify0((inArr-minimosVals)/rangos,normaMat,Labels,3)
    print ("You will probably like this person: " + classifierResult)
    


In [ ]:
classifyPerson()